In [49]:
import cv2
import io
import PIL.Image
import numpy as np
from ipywidgets import Image, VBox, HBox, Widget, Button
from IPython.display import display
from ipywebrtc import CameraStream, ImageRecorder
##########
def compress_to_bytes(data, fmt):
    """
    Helper function to compress image data via PIL/Pillow.
    """
    buff = io.BytesIO()
    img = PIL.Image.fromarray(data)    
    img.save(buff, format=fmt)
    return buff.getvalue()
##########
VIDEO_WIDTH = 640
VIDEO_HEIGHT = 480

camera = CameraStream(constraints={'facing_mode': 'user','audio': False,'video': { 'width': VIDEO_WIDTH, 'height': VIDEO_HEIGHT}})
image_recorder = ImageRecorder(stream=camera)
out = Image(width=VIDEO_WIDTH, height=VIDEO_HEIGHT)

FLAG_STOP = False

def cap_image(_):
    print("coucou")
    if FLAG_STOP:
        return
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)[..., :3]
    canvas = process(im_array)
    ####
    canvas = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    canvas = cv2.GaussianBlur(canvas, (5, 5), 0)
    canvas = cv2.flip(canvas, 0)
    ####
    im_out = PIL.Image.fromarray(canvas)
    f = io.BytesIO()
    im_out.save(f, format='png')
    out.value = f.getvalue()
    image_recorder.recording = True

image_recorder.image.observe(cap_image, names=['value'])


btn_stop = Button(description="Stop",tooltip='click this to stop webcam',button_style='danger')
# btn_stop
def close_cam(_):
    FLAG_STOP= True
    Widget.close_all()
    az = type(image_recorder)
    print("type de : image_recorder = ",az)
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    #print(im_in)
    ae= type(im_in)
    print("type après passage dans PIL.Image.open(io.BytesIO(image_recorder.image.value)) = ",ae)
    im_array = np.array(im_in)#[..., :3]
    #print(im_array)
    ar = type(im_array)
    print("type après passage dans np.array(im_in) = ",ar)
    img_rgb = cv2.flip(im_array, 0)
    fmt = 'png'
    data_comp = compress_to_bytes(img_rgb, fmt)
    at = type(data_comp)
    print("type après passage dans compress_to_bytes(img_rgb, fmt) = ",at)
    image_headline = Image(width=VIDEO_WIDTH, height=VIDEO_HEIGHT)
    image_headline.value = data_comp
    print(type(image_headline))
    display(image_headline)
    
    #
btn_stop.on_click(close_cam)
# Run this section and Press the Camera button to display demo
display(VBox([HBox([camera, image_recorder, btn_stop])]))   #  btn_stop]), out]))


type de : image_recorder =  <class 'ipywebrtc.webrtc.ImageRecorder'>
type après passage dans PIL.Image.open(io.BytesIO(image_recorder.image.value)) =  <class 'PIL.PngImagePlugin.PngImageFile'>
type après passage dans np.array(im_in) =  <class 'numpy.ndarray'>
type après passage dans compress_to_bytes(img_rgb, fmt) =  <class 'bytes'>
<class 'ipywidgets.widgets.widget_media.Image'>


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x06\x00\x00\x005\xd1\xdc…